# 05 - 3D Trajectory Reconstruction

Module 4: From 2D video observations to 3D trajectory, bounce detection, and in/out classification.

Knowledge applied:
- Homography (2D -> court projection)
- Extended Kalman Filter (6D nonlinear state estimation)
- Physics modeling (projectile motion under gravity)
- Machine Learning (bounce classification)

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import sys, os
sys.path.insert(0, os.path.abspath('..'))

from src.trajectory_3d import (
    CourtProjector,
    PhysicsModel,
    ExtendedKalmanFilter3D,
    BounceDetector,
    InOutClassifier,
    TrajectoryReconstructor,
    Trajectory3D,
    TrajectoryPoint3D,
    BounceEvent,
)

%matplotlib inline
plt.rcParams['figure.figsize'] = (14, 8)

## 1. Simulated 3D Trajectory

Create a realistic 3D ball trajectory with physics (gravity + bounce).

In [ ]:
# Simulate a ball trajectory: serve from baseline to opposite court
np.random.seed(42)
fps = 30
g = 9.81

# Initial conditions (meters, m/s)
x0, y0, z0 = 1.0, 5.5, 2.5  # serve position
vx0, vy0, vz0 = 15.0, 0.5, 3.0  # initial velocity

# Generate trajectory with bounce
dt = 1.0 / fps
positions = []
x, y, z = x0, y0, z0
vx, vy, vz = vx0, vy0, vz0
cor = 0.75  # coefficient of restitution

for i in range(90):  # 3 seconds
    positions.append((x, y, z, vx, vy, vz))
    
    x += vx * dt
    y += vy * dt
    z += vz * dt - 0.5 * g * dt**2
    vz -= g * dt
    
    # Bounce when z < 0
    if z < 0:
        z = 0.01
        vz = -vz * cor
        vx *= 0.9  # friction
        vy *= 0.9

pos_array = np.array(positions)

# Plot 3D trajectory
fig = plt.figure(figsize=(14, 8))
ax = fig.add_subplot(111, projection='3d')
ax.plot(pos_array[:, 0], pos_array[:, 1], pos_array[:, 2], 'b-', linewidth=2)

# Mark bounce points
for i in range(1, len(pos_array)):
    if pos_array[i-1, 2] > 0.1 and pos_array[i, 2] < 0.1:
        ax.scatter([pos_array[i, 0]], [pos_array[i, 1]], [0],
                   c='red', s=100, marker='o', label='Bounce' if i < 30 else '')

ax.set_xlabel('X (m) - Court Length')
ax.set_ylabel('Y (m) - Court Width')
ax.set_zlabel('Z (m) - Height')
ax.set_title('Simulated 3D Ball Trajectory')
ax.legend()
plt.show()

print(f'Trajectory: {len(positions)} frames ({len(positions)/fps:.1f}s)')
print(f'Start: ({x0:.1f}, {y0:.1f}, {z0:.1f}) m')
print(f'Max height: {pos_array[:, 2].max():.2f} m')

## 2. Extended Kalman Filter 3D Tracking

In [ ]:
# Run EKF on the simulated trajectory (observing only x, y)
ekf = ExtendedKalmanFilter3D(dt=dt, gravity=g)

# Add measurement noise to 2D observations
noise_std = 0.1  # meters
obs_x = pos_array[:, 0] + np.random.randn(len(pos_array)) * noise_std
obs_y = pos_array[:, 1] + np.random.randn(len(pos_array)) * noise_std

ekf.initialize(obs_x[0], obs_y[0], z=2.5, vx=15.0, vy=0.5, vz=3.0)

ekf_states = []
for i in range(len(pos_array)):
    ekf.predict()
    state = ekf.update(obs_x[i], obs_y[i])
    ekf_states.append(state.copy())

ekf_states = np.array(ekf_states)

# Compare EKF vs Ground Truth
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

labels = ['X (length)', 'Y (width)', 'Z (height)']
for i in range(3):
    t_frames = np.arange(len(pos_array)) / fps
    axes[i].plot(t_frames, pos_array[:, i], 'g-', label='Ground Truth', linewidth=2)
    axes[i].plot(t_frames, ekf_states[:, i], 'b--', label='EKF Estimate', linewidth=2)
    if i < 2:
        axes[i].scatter(t_frames, [obs_x, obs_y][i], c='red', s=5, alpha=0.5, label='Observations')
    axes[i].set_xlabel('Time (s)')
    axes[i].set_ylabel(f'{labels[i]} (m)')
    axes[i].set_title(f'{labels[i]} Estimation')
    axes[i].legend()

plt.tight_layout()
plt.show()

# Height estimation error
z_error = np.abs(ekf_states[:, 2] - pos_array[:, 2])
print(f'Height estimation - Mean error: {z_error.mean():.3f} m, Max: {z_error.max():.3f} m')

## 3. Bounce Detection

In [ ]:
# Create trajectory object from EKF results
trajectory = Trajectory3D()
for i, state in enumerate(ekf_states):
    point = TrajectoryPoint3D(
        frame_id=i,
        x=state[0], y=state[1], z=max(0, state[2]),
        vx=state[3], vy=state[4], vz=state[5],
    )
    trajectory.points.append(point)

# Detect bounces
bounce_detector = BounceDetector(
    z_threshold=0.15,
    min_frames_between_bounces=5,
)
bounces = bounce_detector.detect_bounces_physics(trajectory)

print(f'Detected {len(bounces)} bounce events:')
for i, b in enumerate(bounces):
    print(f'  Bounce {i+1}: frame {b.frame_id}, position ({b.x:.2f}, {b.y:.2f}) m')

# Visualize bounces on trajectory
fig, ax = plt.subplots(figsize=(14, 5))
t_frames = np.arange(len(ekf_states)) / fps
ax.plot(t_frames, ekf_states[:, 2], 'b-', label='Estimated Height')
ax.plot(t_frames, pos_array[:, 2], 'g--', alpha=0.5, label='Ground Truth Height')

for b in bounces:
    ax.axvline(x=b.frame_id/fps, color='red', linestyle='--', alpha=0.7)
    ax.scatter([b.frame_id/fps], [0], c='red', s=100, zorder=5, label='Bounce')

ax.set_xlabel('Time (s)')
ax.set_ylabel('Height (m)')
ax.set_title('Bounce Detection from Height Profile')
ax.legend()
plt.show()

## 4. In/Out Classification

In [ ]:
# Classify bounce events
classifier = InOutClassifier(court_type='tennis_singles')

for i, bounce in enumerate(bounces):
    is_in, confidence = classifier.classify_with_confidence(bounce)
    bounce.is_in = is_in
    bounce.confidence = confidence
    
    status = 'IN' if is_in else 'OUT'
    print(f'Bounce {i+1}: ({bounce.x:.2f}, {bounce.y:.2f}) -> {status} (confidence: {confidence:.2f})')

# Visualize on court map
fig, ax = plt.subplots(figsize=(12, 6))

# Draw court
court = plt.Rectangle((0, 0), 23.77, 10.97, fill=False, edgecolor='white', linewidth=2)
singles = plt.Rectangle((0, 1.37), 23.77, 8.23, fill=False, edgecolor='white', linewidth=1, linestyle='--')
ax.add_patch(court)
ax.add_patch(singles)
ax.axvline(x=11.885, color='white', linewidth=2)  # net

# Plot trajectory on court
ax.plot(ekf_states[:, 0], ekf_states[:, 1], 'b-', alpha=0.5, label='Ball trajectory')

# Plot bounces
for b in bounces:
    color = 'green' if b.is_in else 'red'
    marker = 'o' if b.is_in else 'x'
    label = 'IN' if b.is_in else 'OUT'
    ax.scatter([b.x], [b.y], c=color, s=200, marker=marker, zorder=5,
               edgecolors='white', linewidths=2, label=label)

ax.set_facecolor('#2d5a27')
ax.set_xlim(-2, 26)
ax.set_ylim(-2, 13)
ax.set_xlabel('Court Length (m)')
ax.set_ylabel('Court Width (m)')
ax.set_title('Bounce Locations & In/Out Classification')
ax.set_aspect('equal')
ax.legend()
plt.show()

## 5. Full Trajectory Reconstruction Pipeline

In [ ]:
# Demonstrate the full TrajectoryReconstructor
# Using identity homography for this demo (pixel coords = court coords)
H = np.eye(3)  # In practice, this comes from court detection

reconstructor = TrajectoryReconstructor(
    homography=H,
    fps=fps,
    court_type='tennis_singles',
)

# Feed observations
for i in range(len(obs_x)):
    point = reconstructor.process_frame(i, obs_x[i], obs_y[i])

# Finalize and get results
result_trajectory = reconstructor.finalize()

print(f'Trajectory: {len(result_trajectory.points)} points')
print(f'Bounces detected: {len(result_trajectory.bounces)}')
for b in result_trajectory.bounces:
    status = 'IN' if b.is_in else 'OUT'
    print(f'  Frame {b.frame_id}: ({b.x:.2f}, {b.y:.2f}) -> {status} (conf: {b.confidence:.2f})')